In [ ]:
import pandas as pd
from scipy.sparse import lil_matrix
from tqdm import tqdm
from math import log
from collections import Counter

In [3]:
df = pd.read_csv("data/data.csv")

In [4]:
df.head()

,text,topic
0,Турист из России проник на территорию турецкой...,Путешествия
1,Японии удастся разрешить территориальный вопро...,Мир
2,Новый лидер «Талибана» Акхтар Мохаммад Мансур ...,Мир
3,Порошковый алкоголь поступит в продажу в США э...,Из жизни
4,"Автомобиль, использовавшийся командой американ...",Спорт


In [5]:
term_doc_frequency = {}

In [6]:
# для каждого уникального слова в корпусе рассчитаем то, в скольких документах оно встречается
# заодно мы получаем вокабуляр/словарь нашего корпуса текстов, так как ключами term_doc_frequency будут все уникальные слова в корпусе

for text in tqdm(df.text.to_list()):
    words = set(text.lower().split())
    for w in words:
        term_doc_frequency[w] = term_doc_frequency.get(w, 0) + 1

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 25419.67it/s]


In [7]:
# каждому уникальному слову присвоим индекс
# потом эти индексы станут номерами "колонок" в нашей матрице tf-idf, чтобы для каждого слова информация хранилась в одной конкретной колонке

word2index = {}

for w in tqdm(term_doc_frequency):
    word2index[w] = len(word2index)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 225926/225926 [00:00<00:00, 4659590.72it/s]


In [7]:
len(term_doc_frequency)

225926

In [25]:
# создадим sparse матрицу (матрицу, где мы не храним в памяти нули) размерности 
# кол-во документов в нашем корпусе на кол-во уникальных слов в корпусе

tfidf_matrix = lil_matrix((len(df), len(term_doc_frequency)))

In [26]:
# рассчитаем tf-idf матрицу

num_texts = len(df) # сохраняем в переменную кол-во документов, чтобы каждый раз не считать для idf компонента

for i, text in tqdm(enumerate(df.text.to_list()), total=len(df)):
    words = Counter(text.lower().split()) # считаем кол-во (абсолютное) упоминания каждого слова в конкретном тексте
    n = sum(words.values()) # считаем кол-во слов в тексте для знаменателя 
    for w, freq in words.items():
        # рассчитываем и вставляем значения tf-idf для конкретного слова в конкретном тексте
        tfidf_matrix[i, word2index[w]] = freq / n * log(num_texts / term_doc_frequency[w] + 1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:02<00:00, 4661.97it/s]


In [23]:
tfidf_matrix

<List of Lists sparse matrix of dtype 'float64'
	with 1480685 stored elements and shape (10000, 225926)>